In [39]:
import json
import pandas as pd
import numpy as np
# データ可視化ライブラリ
import matplotlib.pyplot as plt
%matplotlib inline  
import pandas_profiling as pdp
import seaborn as sns

In [40]:
train = pd.read_csv('../data/raw/train2.csv')
test = pd.read_csv('../data/raw/test2.csv')

data = pd.concat((train, test))

In [41]:
data.query('Area > 800')

,Type,Region,MunicipalityCode,TimeToNearestStation,FloorPlan,Area,LandShape,Frontage,TotalFloorArea,BuildingYear,...,Housing,office,store,other_use,Warehouse,parking,plant,shareHouse,Workshop,y
77,2,2,13101,1,0,1200,5,22.600000,2500,80,...,0,1,0,0,0,0,0,0,0,4800.0
793,2,2,13101,4,0,1700,5,9.702674,2500,85,...,0,1,0,0,0,0,0,0,0,8100.0
811,2,1,13101,4,0,870,4,24.000000,2500,73,...,1,1,1,0,0,0,0,0,0,3100.0
813,2,2,13101,6,0,1000,5,31.600000,2500,80,...,0,1,0,0,0,0,0,0,0,8700.0
814,2,2,13101,6,0,3100,9,41.500000,2500,95,...,0,1,1,0,0,0,0,0,0,7000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34827,1,3,13401,10,0,1600,1,9.702674,177,97,...,0,0,0,0,0,0,0,0,0,NaN
34831,3,0,13401,10,0,2200,0,9.702674,177,97,...,0,0,0,0,0,0,0,0,0,NaN
34835,1,3,13401,10,0,3100,2,60.000000,177,97,...,0,0,0,0,0,0,0,0,0,NaN
34836,1,3,13401,10,0,1500,5,9.702674,177,97,...,0,0,0,0,0,0,0,0,0,NaN


In [42]:
train.columns

Index(['Type', 'Region', 'MunicipalityCode', 'TimeToNearestStation',
       'FloorPlan', 'Area', 'LandShape', 'Frontage', 'TotalFloorArea',
       'BuildingYear', 'Structure', 'Use', 'Purpose', 'Direction',
       'Classification', 'Breadth', 'CityPlanning', 'CoverageRatio',
       'FloorAreaRatio', 'Period', 'Renovation', 'Remarks', 'NearestStation',
       'latitude', 'longitude', 'h31_price', 'SerialNumber', 'h31_27_mean',
       '31-28', 'building_id', 'L', 'D', 'K', 'S', 'R', 'Maisonette',
       'OpenFloor', 'Studio', 'Housing', 'office', 'store', 'other_use',
       'Warehouse', 'parking', 'plant', 'shareHouse', 'Workshop', 'y'],
      dtype='object')

In [21]:
drop_columns = ['Type','Region', 'MunicipalityCode', 'TimeToNearestStation', 'FloorPlan']
train = train.drop(drop_columns, axis=1)

In [22]:
train = train.drop(train.query('y >= 16000').index, axis=0)

In [24]:
len(train.query('Area == 3100'))

1197

## Area400以上 and MunicipalityCodeが13117以下でモデル作って学習
これに実際に流すのはtest.query('Area > 800 and MunicipalityCode < 13117')

In [85]:
train.query('Area > 400 and MunicipalityCode < 13117')

,Type,Region,MunicipalityCode,TimeToNearestStation,FloorPlan,Area,LandShape,Frontage,TotalFloorArea,BuildingYear,...,Housing,office,store,other_use,Warehouse,parking,plant,shareHouse,Workshop,y
77,2,2,13101,1,0,1200,5,22.600000,2500,80,...,0,1,0,0,0,0,0,0,0,4800.0
271,2,2,13101,3,0,480,4,16.000000,2500,60,...,0,1,0,0,1,0,0,0,0,2900.0
300,2,2,13101,5,0,510,5,9.702674,1800,93,...,0,1,0,0,0,0,0,0,0,2100.0
412,2,2,13101,4,0,650,4,18.500000,2500,91,...,0,1,0,0,0,0,0,0,0,2900.0
420,2,2,13101,3,0,500,5,20.000000,2500,64,...,0,1,0,0,1,0,0,0,1,390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262964,2,2,13116,7,0,410,5,17.000000,1100,96,...,0,1,0,0,0,1,0,0,0,510.0
263172,1,1,13116,5,0,500,4,17.000000,177,97,...,0,0,0,0,0,0,0,0,0,200.0
263292,2,1,13116,5,0,690,5,22.700000,530,92,...,1,0,0,0,0,0,0,1,0,310.0
263336,2,2,13116,4,0,580,5,20.000000,1500,89,...,0,0,0,0,0,0,0,1,0,410.0


In [86]:
train.query('y > 10000')['MunicipalityCode'].describe()

count       46.000000
mean     13108.500000
std         15.239933
min      13101.000000
25%      13102.250000
50%      13103.500000
75%      13110.750000
max      13204.000000
Name: MunicipalityCode, dtype: float64

In [65]:
train.query('Area > 3000 and (Type ==1 or Type ==2) and  MunicipalityCode < 13117 and FloorPlan == 0 and (TotalFloorArea == 177 \
          or TotalFloorArea == 2500) and (Structure == 0 or Structure == 23) and \
           (Classification == 14 or Classification == 2) and building_id == -1')['y'].describe()



count      182.000000
mean      4173.626374
std       6360.838146
min        200.000000
25%       1100.000000
50%       2200.000000
75%       4400.000000
max      61000.000000
Name: y, dtype: float64

In [76]:
test.query('Area > 800 and MunicipalityCode < 13117')

,Type,Region,MunicipalityCode,TimeToNearestStation,FloorPlan,Area,LandShape,Frontage,TotalFloorArea,BuildingYear,...,Studio,Housing,office,store,other_use,Warehouse,parking,plant,shareHouse,Workshop
1305,2,2,13103,1,0,3100,5,60.0,2500,101,...,0,0,1,0,0,0,0,0,0,0
1739,2,1,13103,3,0,1000,9,28.0,2500,95,...,0,0,1,0,0,0,0,0,1,0
1868,2,2,13103,4,0,850,2,37.9,2500,93,...,0,0,1,0,0,0,1,0,1,0
1960,1,2,13103,4,0,3100,1,60.0,177,97,...,0,0,0,0,0,0,0,0,0,0
2295,2,2,13104,3,0,1800,5,40.0,2500,73,...,0,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16268,1,1,13115,3,0,1700,5,40.0,177,97,...,0,0,0,0,0,0,0,0,0,0
16296,1,1,13115,3,0,1600,4,46.9,177,97,...,0,0,0,0,0,0,0,0,0,0
16447,2,1,13115,16,0,1600,5,15.0,1600,63,...,0,0,0,0,1,0,0,0,0,0
16505,1,1,13115,11,0,870,9,27.0,177,97,...,0,0,0,0,0,0,0,0,0,0


Type: 1 or 2　　
Region: 1 or 2, 2.3個の4　　
MunicipalityCode: 13116以下がほとんど　　
FloorPlan: 全て0　　
Areaほとんど3100　　
TotalFloorArea: 177 2500　　
Structure 0 23　　
Classification 14 2　　
building_id -1　　